# Project

1. Для понимания эффективности новой фичи нужно посчитать показатели Retention 7 дня  и ARPPU за два периода: до и после внедрения фичи

2. Сформировать выводы по анализу

1. Считаем Retention 7 дня за период до релиза (май 2019). Чтобы посчитать retention не для одной даты, а для всего периода, необходимо разделить сумму по столбцу "7 день" для всех входящих в период когорт и разделить на сумму по столбцу "зарегистрировано клиентов" для этих же когорт.

 ! pip install psycopg2 

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

Создаем датафрейм с retention за весь период (до и после релиза) и сразу рассчитываем lifetime для каждой строки

In [3]:
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count = pd.DataFrame(getRegisteredUsersCount())
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])
retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int) # Приведем тип к целому числу
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-05-01,2019-05-01,2590,6516,0
1,2019-05-02,2019-05-01,2146,6516,1
2,2019-05-02,2019-05-02,2049,5309,0
3,2019-05-03,2019-05-01,1637,6516,2
4,2019-05-03,2019-05-02,1817,5309,1


Добавляем к этой таблице данные по доходам

In [4]:
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())

In [5]:
for col in ['date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])

In [6]:
revenue = revenue.rename(columns={'date':'activity_date'})
retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672.0,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420.0,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367.0,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048.0,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164.0,533.608


In [7]:
for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164,533.608


Теперь у нас есть общий датафрейм, можем выполнять расчеты. Фильтруем таблицу по retention 7 дня для пользователей, привлеченных до релиза (май 2019)

In [8]:
retention_7_may_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"]
retention_7_may_table.head()

<ipython-input-8-7f2647d2bdcd>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_may_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"]


,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
28,2019-05-08,2019-05-01,953,6516,7,616,63.9065
37,2019-05-09,2019-05-02,745,5309,7,490,313.7220
47,2019-05-10,2019-05-03,895,6266,7,571,110.7260
58,2019-05-11,2019-05-04,1186,8639,7,750,124.8590
70,2019-05-12,2019-05-05,1358,9787,7,885,174.9240


Выполняем расчет retention 7 дня для этих когорт

In [9]:
retention_7_may = retention_7_may_table['active_users_count'].sum() / retention_7_may_table['registered_users_count'].sum()
print(retention_7_may)

0.12933000957960866


Получается, что retention 7 дня для пользователей, привлеченных до релиза = 12,9%
Теперь делаем аналогичный расчет для июня

In [10]:
retention_7_june_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"]
retention_7_june_table.head()

<ipython-input-10-d06bb8139afa>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  retention_7_june_table = retention_table_with_revenue[retention_table_with_revenue['lifetime']==7][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"]


,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
736,2019-06-08,2019-06-01,545,4833,7,365,176.3160
767,2019-06-09,2019-06-02,683,5255,7,461,193.8390
798,2019-06-10,2019-06-03,484,4193,7,320,27.7548
829,2019-06-11,2019-06-04,511,4194,7,320,54.9610
860,2019-06-12,2019-06-05,472,3998,7,301,63.1554


In [11]:
retention_7_june = retention_7_june_table['active_users_count'].sum() / retention_7_june_table['registered_users_count'].sum()
print(retention_7_june)

0.12560867530346634


Retention 7 дня в июне составляет 12,6%. Это незначительно, но все же ниже, чем в период до релиза. 
Теперь расчитаем ARPPU до и после (считаем средний доход именно на платящих клиентов, поэтому делить будем на 'users_count_with_revenue')

In [12]:
may_arppu = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"].sum() / retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-05-01"][retention_table_with_revenue['registration_date']<"2019-06-01"].sum()
print(may_arppu)

0.30612453524872973


В мае, до релиза, ARPPU = 0,3 у.е. Теперь считаем июнь:

In [13]:
june_arppu = retention_table_with_revenue['revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"].sum() / retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>="2019-06-01"][retention_table_with_revenue['registration_date']<"2019-07-01"].sum()
print(june_arppu)

0.39319398670206207


Похоже, что в июне средний доход с 1 платящего клиента вырос относительно мая и составляет 0,4 у.е.

Выводы:
* Retention 7 дня остался примерно на прежнем уровне
* Тем не менее ARPPU в июне вырос на 30%

Возможно, на ARPPU могли повлиять и другие факторы. Чтобы убедиться в этом, необходимо было бы провести раскатку функционала на часть пользователей и/или провести анализ с учетом фактора использования конкретно нашей фичи, однако таких данных, как это часто бывает, у нас нет. Здесь явно имеет смысл настроить сбор таких данных на будущее, но на текущий момент и в текущих условиях можно сказать, что наша фича оказалась вполне эффективна. Во всяком случае, ARPPU она не обрушила, а значит можно оставлять ее на проде.  